In [6]:
import math
import os
import glob
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
from pandas import json_normalize
import yaml
from yaml import CSafeLoader as Loader

sns.set_context("paper", font_scale=3)

LOG_DIR = os.path.expanduser("~/lis-cluster/emergent_communication/emergent_communication/lightning_logs/")


In [ ]:
results_df = []
for run_dir in tqdm(os.listdir(LOG_DIR)):
    run_path = os.path.join(LOG_DIR, run_dir, "checkpoints/")

    results = glob.glob(run_path+"*.pickle")
    for result in results:
        df = pd.read_pickle(result)
        df = pd.DataFrame.from_records(df)
        df["dir_name"] = run_dir
        df["epoch"] = int(result.split("epoch=")[1].split("-")[0])
        results_df.append(df)


results_df = pd.concat(results_df, ignore_index=True)

# Remove superfluous NaN cells
def compress(values):
    for val in values:
        if val is not None and not np.isnan(val):
            return val

results_df = results_df.groupby(["dir_name", "epoch"]).aggregate(compress)
results_df.reset_index(inplace=True)
results_df

 44%|████▍     | 346/780 [00:43<01:55,  3.75it/s]

In [5]:
hp = []
for run_dir in tqdm(os.listdir(LOG_DIR)):
    file_path = os.path.join(LOG_DIR, run_dir, "hparams.yaml")
    file = yaml.load(open(file_path), Loader=Loader) #safe_load(, Loader=Loader)
    df = json_normalize(file)
    df["dir_name"] = run_dir
    hp.append(df)

hp = pd.concat(hp, ignore_index=True)
hp

100%|██████████| 780/780 [00:20<00:00, 38.88it/s]


,accelerator,accumulate_grad_batches,amp_backend,amp_level,auto_lr_find,auto_scale_batch_size,auto_select_gpus,baseline_type,batch_size,benchmark,...,tpu_cores,track_grad_norm,val_check_interval,vocab_size,vocab_size_feedback,weights_save_path,weights_summary,dir_name,receiver_layer_norm,sender_layer_norm
0,None,None,native,None,False,False,False,mean,5120,None,...,None,-1,None,5,3,None,top,version_1060567,NaN,NaN
1,None,None,native,None,False,False,False,mean,5120,None,...,None,-1,None,5,2,None,top,version_1060572,NaN,NaN
2,None,None,native,None,False,False,False,mean,5120,None,...,None,-1,None,5,2,None,top,version_1060573,NaN,NaN
3,None,None,native,None,False,False,False,mean,5120,None,...,None,-1,None,5,3,None,top,version_1060570,NaN,NaN
4,None,None,native,None,False,False,False,mean,5120,None,...,None,-1,None,5,2,None,top,version_1060568,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
775,None,None,native,None,False,False,False,mean,5120,None,...,None,-1,None,100,2,None,top,version_1073800,True,True
776,None,None,native,None,False,False,False,mean,5120,None,...,None,-1,None,100,3,None,top,version_1073799,True,True
777,None,None,native,None,False,False,False,mean,5120,None,...,None,-1,None,100,3,None,top,version_1073795,True,True
778,None,None,native,None,False,False,False,mean,5120,None,...,None,-1,None,100,2,None,top,version_1073796,True,True


In [ ]:
def fix_duplicate_value(val, allow_offset=None):
    if isinstance(val, list):
        for el in val:
            if allow_offset is None:
                assert (el == val[0]) or (el == "None") or (val[0] == "None")
            else:
                assert (np.abs(el - val[0]) < allow_offset) or (el == "None") or (val[0] == "None")
        return val[0]
    else:
        return val

hp = hp.applymap(fix_duplicate_value)

In [ ]:
# TODO:
# REFERENCE_METRIC = "val_acc_no_noise"
REFERENCE_METRIC = "val_acc"


indices_best_steps = results_df.groupby("dir_name")[REFERENCE_METRIC].idxmax()

df = results_df.loc[list(indices_best_steps)].copy()
df

In [ ]:
df.set_index("dir_name", inplace=True, drop=False)
if not hp.index.name == "dir_name":
    hp.set_index("dir_name", inplace=True, verify_integrity=True)
df = df.join(hp, how="left")

In [ ]:
MAX_N_RUNS = 10
df.dropna(subset=["num_attributes", "num_values"], inplace=True)
df.fillna({"sender_layer_norm": 0, "receiver_layer_norm": 0}, inplace=True)

assert (df.sender_entropy_coeff == df.receiver_entropy_coeff).all()
assert (df.num_senders == df.num_receivers).all()
assert (df.sender_layer_norm == df.receiver_layer_norm).all()

df["entropy_coeff"] = df["sender_entropy_coeff"]
df["num_agents"] = df["num_senders"]
df["layer_norm"] = df["sender_layer_norm"]

runs_best_entropy = []

df["attr_val"] = df["num_attributes"].map(int).map(str) + "_" + df["num_values"].map(int).map(str)
attr_val_combinations = df["attr_val"].unique()

num_agents_values = df["num_agents"].unique()
for num_agents in num_agents_values:
    print(f"\nNum agents: {num_agents}")

    for attr_val in attr_val_combinations:
        n_attributes = int(float(attr_val.split("_")[0]))
        n_values = int(float(attr_val.split("_")[1]))
        print(f"\n\t\tAttr: {n_attributes} Values: {n_values}")

        # length_cost_values = df["length_cost"].unique()
        length_cost_values = [0, 0.001]
        for length_cost in length_cost_values:
            for layer_norm in [0, 1]:
                max_len_values = df.max_len.unique()
                for max_len in max_len_values:
                    vocab_size_values = df.vocab_size.unique()
                    for vocab_size in vocab_size_values:
                        noise_values = df.noise.unique()
                        for noise in noise_values:
                            for feedback in (0, 1):
                                for self_repair in (0, 1):

                                    df_config = df[(df.attr_val == attr_val) & (df.length_cost == length_cost) & (df.feedback == feedback) & (df.num_agents == num_agents) & (df.noise == noise) & (df.self_repair == self_repair) & (df.max_len == max_len) & (df.vocab_size == vocab_size) & (df.layer_norm == layer_norm)]

                                    if len(df_config) == 0:
                                        continue

                                    print(f"\t\t\tLength cost: {length_cost}\t Noise: {noise}\tmax_len: {max_len}\t vocab_size: {vocab_size}\t layer_norm: {layer_norm}", end="")

                                    if feedback:
                                        print("\tFeedback", end="")
                                    elif self_repair:
                                        print("\tSelf repair", end="")
                                    else:
                                        print("\tBaseline", end="")

                                    print(f"\tFound {len(df_config)} runs") #: {df_config}

                                    avg_val_accs = df_config.groupby("sender_entropy_coeff").aggregate({REFERENCE_METRIC: "mean"})
                                    # Take the highest entropy coeff in case of tie
                                    best_entropy_coeff = avg_val_accs[avg_val_accs[REFERENCE_METRIC] == avg_val_accs.max()[0]].index[-1]

                                    df_best_entropy = df_config[df_config.entropy_coeff == best_entropy_coeff]
                                    print(f"\t\t\t\t\t\tbest entropy coeff: {best_entropy_coeff}; num runs: {len(df_best_entropy)}", end="")
                                    print(f"\tother: {avg_val_accs.to_dict()})", )

                                    if len(df_best_entropy) > MAX_N_RUNS:
                                        df_best_entropy = df_best_entropy.tail(10)
                                    runs_best_entropy.append(df_best_entropy)

data = pd.concat(runs_best_entropy, ignore_index=True)




In [ ]:
def calc_capacity(row):
    return math.pow(row.num_values, row.num_attributes)

data["capacity"] = data.apply(calc_capacity, axis=1)
data.sort_values("capacity", inplace=True)

In [ ]:
data["condition"] = data["noise"].map(lambda x: f"noise_{x}" if x > 0 else "baseline") + data["length_cost"].map(lambda x: f"_length_cost_{x}" if x > 0 else "") + data["feedback"].map(lambda x: "_feedback" if x else "") + data["self_repair"].map(lambda x: "_self_repair" if x else "")

target_data = data.copy()

NUM_AGENTS = 1
target_data = target_data[target_data.num_agents == NUM_AGENTS]

MAX_LEN = 5
target_data = target_data[target_data.max_len == MAX_LEN]

VOCAB_SIZE = 100
target_data = target_data[target_data.vocab_size == VOCAB_SIZE]

LAYER_NORM = 1
target_data = target_data[target_data.layer_norm == LAYER_NORM]


# ATTR_VAL = "4_5"
# target_data = target_data[(target_data["attr_val"] == ATTR_VAL)]

# print(target_data.dir_name.unique())

# hue_order = None
# hue_order = ["baseline", "noise_0.1", "noise_0.1_feedback_binary"]
hue_order = ["baseline", "noise_0.1", "noise_0.1_self_repair", "noise_0.1_feedback"]

# order = ["2_10", "3_5", "2_16", "4_4", "4_5", "3_10", "2_100"]
order = ["2_16"]


target_hparam = "attr_val"

num_runs_data = target_data.groupby(["attr_val", "condition"]).size().reset_index()
# plt.figure(figsize=(30, 10))

_, axes = plt.subplots(10, 1, figsize=(50, 150))

sns.boxplot(ax=axes[0], data=num_runs_data, x="attr_val", order=order, hue="condition", hue_order=hue_order, y=0)

# next_axis_idx = (0, 0)
for i, metric in enumerate(["val_acc", "test_acc_no_noise", "topsim", "posdis", "bosdis", "test_acc", "val_acc_no_noise", "train_acc_no_noise", "sender_entropy_coeff"]):
    sns.boxplot(ax=axes[i+1], data=target_data, x=target_hparam, y=metric, order=order, hue="condition", hue_order=hue_order, boxprops=dict(alpha=.5), showfliers = False)
    ax = sns.swarmplot(ax=axes[i+1], data=target_data, x=target_hparam, y=metric, order=order, hue="condition", hue_order=hue_order, dodge=True)
    handles, labels = ax.get_legend_handles_labels()
    num_conditions = int(len(handles)/2) if not hue_order else len(hue_order)
    ax.legend(handles[:num_conditions], labels[:num_conditions])
    # plt.setp(ax.get_legend().get_texts(), fontsize='22')
    # if next_axis_idx[1] >= axes.shape[1]-1:
    #     next_axis_idx = (next_axis_idx[0]+1, 0)
    # else:
    #     next_axis_idx = (next_axis_idx[0], next_axis_idx[1]+1)

name = "results"
plt.savefig("plots/"+name+".pdf", dpi=300)